In [2]:
import pandas as pd
import numpy as np

Загружаем датасет

In [3]:
data_orig = pd.read_csv('Titanic.csv')
data_orig

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


До обработки размер датасета

In [4]:
data_orig.shape

(891, 12)

Удаляем все строки, содержащие пропущенные данные

In [5]:
data_proc = data_orig.dropna()

Посмотрим какие типы данных имеют признаки

In [6]:
data_proc.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

Видим, что признаки Name, Sex, Ticket, Cabin и Embarked имеют не числовые значения. Удаляем все, кроме Sex и Embarked

In [7]:
data_proc = data_proc.drop(["Name", "Ticket","Cabin"], axis=1)

Выведем уникальные значения в оставшихся категориальных столбцах Sex и Embarked

In [8]:
print("Sex:", data_proc["Sex"].unique())
print("Embarked:", data_proc["Embarked"].unique())

Sex: ['female' 'male']
Embarked: ['C' 'S' 'Q']


Кодируем категориальные данные в столбцах Sex и Embarked

In [9]:
data_proc.loc[data_proc["Sex"] == "male", "Sex"] = 0
data_proc.loc[data_proc["Sex"] == "female", "Sex"] = 1
data_proc["Sex"] = pd.to_numeric(data_proc["Sex"])

In [10]:
data_proc.loc[data_proc["Embarked"] == "C", "Embarked"] = 1
data_proc.loc[data_proc["Embarked"] == "S", "Embarked"] = 2
data_proc.loc[data_proc["Embarked"] == "Q", "Embarked"] = 3
data_proc["Embarked"] = pd.to_numeric(data_proc["Embarked"])

Удалим столбец PassangerId

In [11]:
data_proc.drop(["PassengerId"], axis=1, inplace=True)

Посмотрим сколько данных у нас осталось после обработки

In [12]:
data_proc.shape

(183, 8)

In [13]:
print("Процент оставшихся данных:", data_proc.shape[0] * data_proc.shape[1] / data_orig.shape[0] / data_orig.shape[1] * 100, "%")

Процент оставшихся данных: 13.692480359147027 %


Таким образом мы потеряли примерно 86.3% данных

Далее отделим целевую переменную от данных и разобъем их на обучающую и тестовую выборки

In [14]:
X = data_proc.drop(["Survived"], axis=1)
y = data_proc["Survived"]

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

Применим логистическую регрессию

In [16]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter=200)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

Выведем точность полученной модели

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7272727272727273

Удалим из обучающей и тестовой выборок признак Embarked и обучим на полученных данных логистическую регрессию

In [18]:
X_train2 = X_train.drop(["Embarked"], axis=1)
X_test2 = X_test.drop(["Embarked"], axis=1)

In [19]:
clf = LogisticRegression(random_state=0, max_iter=200)
clf.fit(X_train2, y_train)
y_pred2 = clf.predict(X_test2)
accuracy_score(y_test, y_pred2)

0.6909090909090909

Качество получилось хуже, таким образом признак Embarked сильно влият на точность модели